In [1]:
import requests
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import dotenv_values
import sqlite3
import sqlite_utils
import time
import statistics


In [2]:
mid_kr_puuid = "yke-_ZuAP9wM6s3tpe-FnVWLlnwI4qLuLRzz933HfpI_jzKcDa1YwXbWGpFC7WJek_rv4x_kXk2Y-A"

In [3]:
conn = sqlite3.connect('koren_soloq_db.db')
cursor = conn.cursor()
match_end_data = cursor.execute('''SELECT metadata, info FROM MATCH_DATA''').fetchall()
match_timeline_data = cursor.execute('''SELECT metadata, timelineinfo FROM MATCH_DATA''').fetchall()

conn.close()

In [49]:
#Filter only mid role games
list_match_end_data_whereMid = []
mid_matchs = []
for idx_match, match in enumerate(match_end_data) :
    match_info = json.loads(match[1])
    for idx, participant in enumerate(match_info['participants']):
        if (participant['puuid'] == mid_kr_puuid) and (participant['teamPosition'] == "MIDDLE"):
            list_match_end_data_whereMid.append(match)
            mid_matchs.append(idx_match)

list_match_timeline_data_whereMid = [match_timeline_data[i] for i in mid_matchs]

In [50]:
print(len(match_end_data))
print(len(list_match_end_data_whereMid))
print(len(list_match_timeline_data_whereMid))

600
505
505


In [46]:
#Info after game
kda_list = []
cs_min = []
for match in list_match_end_data_whereMid :


    match_metadata = json.loads(match[0])
    for idx, player_puuid in enumerate(match_metadata['participants']):
        if player_puuid == mid_kr_puuid:
            index_qing = idx

    match_info = json.loads(match[1])

    #KDA
    kills = match_info['participants'][index_qing]['kills']
    assists = match_info['participants'][index_qing]['assists']
    deaths = match_info['participants'][index_qing]['deaths']
    if deaths == 0 : deaths = 1
    kda_list.append((kills+assists)/deaths)

    #CS/MIN
    monster_cs = match_info['participants'][index_qing]['neutralMinionsKilled']
    minion_cs = match_info['participants'][index_qing]['totalMinionsKilled']
    cs_min.append((monster_cs + minion_cs) /  (match_info['gameDuration'] //60))




In [ ]:
#Info timeline
#for match in list_match_timeline_data_whereMid:


# gold diff
# forward %

In [47]:
#attention il a des games autofills
print(statistics.mean(kda_list))
print(statistics.mean(cs_min))

4.438730873087309
7.886367344876574
